In [1]:
from glob import glob

dirs = glob("./data/v1/*") + glob("./data/v1/*")
dirs

['./data/v1/test_total_reward_mean',
 './data/v1/val_total_reward_mean',
 './data/v1/train_episode_average_loss',
 './data/v1/train_loss',
 './data/v1/train_total_reward',
 './data/v1/test_total_reward_mean',
 './data/v1/val_total_reward_mean',
 './data/v1/train_episode_average_loss',
 './data/v1/train_loss',
 './data/v1/train_total_reward']

In [ ]:
from glob import glob
import shutil
import os
from utils import read_yaml

old_dirs = []
new_dirs = []
for foo in glob("./training_results/*"):
    bar = glob(os.path.join(foo, "*/*/*test*"))
    if len(bar) == 0:
        continue
    bar = bar[0]
    old_dir = "/".join(bar.split("/")[:-1])
    old_dirs.append(old_dir)
    hparams = read_yaml(os.path.join(old_dir, "hparams.yaml"))

    des_version = hparams["des_version"]
    allow_random_human = hparams["allow_random_human"]
    allow_random_question = hparams["allow_random_question"]
    pretrain_semantic = hparams["pretrain_semantic"]
    varying_rewards = hparams["varying_rewards"]
    capacity = hparams["capacity"]["episodic"] + hparams["capacity"]["semantic"]
    des_size = hparams["des_size"]
    seed = hparams["seed"]

    new_dir = (
        f"training_results/"
        f"des_version={des_version}_"
        f"allow_random_human={allow_random_human}_"
        f"allow_random_question={allow_random_question}_"
        f"pretrain_semantic={pretrain_semantic}_"
        f"varying_rewards={varying_rewards}_"
        f"des_size={des_size}_"
        f"capacity={capacity}_"
        f"seed={seed}"
    )
    new_dirs.append(new_dir)
    os.rename(old_dir, new_dir)

In [3]:
import logging

logger = logging.getLogger()
logger.disabled = True

from pprint import pprint
from room_env.utils import get_handcrafted


results = get_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=0.1,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
    allow_random_human=False,
    allow_random_question=False,
    varying_rewards=False,
    check_resources=True,
    version="v2",
)
pprint(results)

/home/tk/.virtualenvs/dev-python3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/tk/.virtualenvs/dev-python3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/tk/.virtualenvs/dev-python3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:133: UserWarning: WARN: The obs returned by the `reset()` method should be an int or np.int64, actual type: <class 'dict'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/home/tk/.virtualenvs/dev-python3.8/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: T

capacity=2, strategy=episodic,	pre_sem=False,	rewards_mean=-9.3, rewards_std=3.288
capacity=2, strategy=semantic,	pre_sem=False,	rewards_mean=-7.3, rewards_std=2.83
capacity=2, strategy=random,	pre_sem=False,	rewards_mean=-9.9, rewards_std=3.534
capacity=2, strategy=pre_sem,	pre_sem=True,	rewards_mean=-9.3, rewards_std=4.196

capacity=4, strategy=episodic,	pre_sem=False,	rewards_mean=-7.1, rewards_std=2.773
capacity=4, strategy=semantic,	pre_sem=False,	rewards_mean=-3.3, rewards_std=2.83
capacity=4, strategy=random,	pre_sem=False,	rewards_mean=-6.5, rewards_std=3.722
capacity=4, strategy=pre_sem,	pre_sem=True,	rewards_mean=-7.3, rewards_std=3.288

capacity=8, strategy=episodic,	pre_sem=False,	rewards_mean=-5.1, rewards_std=2.914
capacity=8, strategy=semantic,	pre_sem=False,	rewards_mean=1.1, rewards_std=1.3
capacity=8, strategy=random,	pre_sem=False,	rewards_mean=-3.5, rewards_std=3.263
capacity=8, strategy=pre_sem,	pre_sem=True,	rewards_mean=-2.5, rewards_std=2.5

capacity=16, strateg

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from pprint import pprint
from room_env.utils import get_handcrafted


results = get_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=1.0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
    allow_random_human=False,
    allow_random_question=False,
    varying_rewards=False,
    check_resources=True,
    version="v1",
)
pprint(results)

In [ ]:
from glob import glob
import numpy as np

paths = glob("./training_results_AMD/*/test_debug*")
categories = sorted(
    list(set([path.split("/")[-2].split("-seed")[0] for path in paths]))
)
categories = [foo for foo in categories if "-gamma=0.99-" not in foo]

foo = [
    (
        category,
        np.mean(
            [
                float(path.split("/")[-1].split("-mean=")[-1].split("-std=")[0])
                for path in paths
                if category in path
            ]
        )
        .round(3)
        .item(),
    )
    for category in categories
]
foo

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from glob import glob
import os
from utils import read_yaml

# des_size=l-capacity=2-pretrain=True-gpus=0-seed=0

for path in glob("./training_results_AMD/*/*/version*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    allow_random_human = hparams["allow_random_human"]
    allow_random_question = hparams["allow_random_question"]
    pretrain_semantic = hparams["pretrain_semantic"]
    varying_rewards = hparams["varying_rewards"]
    gamma = hparams["gamma"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]
    dir_name_new = (
        f"./training_results_AMD/des_size={des_size}-capacity={capacity}-"
        f"allow_random_human={allow_random_human}-allow_random_question={allow_random_question}-"
        f"pretrain={pretrain_semantic}-varying_rewards={varying_rewards}-gamma={gamma}-"
        f"gpus={gpus}-seed={seed}"
    )
    print(dir_name_old, dir_name_new)
    os.rename(dir_name_old, dir_name_new)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True
import room_env
from room_env import utils

In [ ]:
from pprint import pprint

for allow_random_human in [True, False]:
    for allow_random_question in [True, False]:

        print(
            f"allow_random_human: {allow_random_human}, allow_random_question: {allow_random_question}"
        )

        results = utils.get_handcrafted(
            env="RoomEnv-v2",
            des_size="l",
            seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            question_prob=0.1,
            policies={
                "memory_management": "rl",
                "question_answer": "episodic_semantic",
                "encoding": "argmax",
            },
            capacities=[32],
            allow_random_human=allow_random_human,
            allow_random_question=allow_random_question,
            varying_rewards=False,
        )
        pprint(results)

In [ ]:
from copy import deepcopy
from glob import glob
from utils import read_yaml, read_json, write_yaml

foo = read_yaml("./train.yaml")

for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [True, False]:
        for seed in [0, 1, 2, 3, 4]:
            foo = deepcopy(foo)
            foo["pretrain_semantic"] = pretrain_semantic
            foo["capacity"] = {
                "episodic": capacity // 2,
                "semantic": capacity // 2,
                "short": 1,
            }
            foo["seed"] = seed
            write_yaml(foo, f"{capacity}_{pretrain_semantic}_{seed}.yaml")

In [ ]:
from glob import glob
import os
import numpy as np
from utils import read_yaml, read_json

results_ = []
for path in glob("./training_results_AMD/*/*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    pretrain_semantic = hparams["pretrain_semantic"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]

    test_mean = float(path.split("test_debug-mean=")[-1].split("-std")[0])
    results_.append((des_size, capacity, pretrain_semantic, seed, test_mean))

results = {}
for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [False, True]:
        results[f"capacity={capacity}_pretrain_semantic={pretrain_semantic}"] = round(
            np.mean(
                [
                    foo[-1]
                    for foo in results_
                    if foo[1] == capacity and foo[2] == pretrain_semantic
                ]
            ),
            3,
        ).item(), round(
            np.std(
                [
                    foo[-1]
                    for foo in results_
                    if foo[1] == capacity and foo[2] == pretrain_semantic
                ]
            ).item(),
            3,
        )
results

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from pprint import pprint
from room_env.utils import get_handcrafted

results = get_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=0.1,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
)
pprint(results)

In [ ]:
import gym
import room_env

capacity = 32
env = gym.make(
    "RoomEnv-v2",
    des_size="l",
    seed=0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacity={"episodic": capacity // 2, "semantic": capacity // 2},
    question_prob=0.1,
    observation_params="perfect",
    allow_random_human=True,
    allow_random_question=True,
    pretrain_semantic=False,
    check_resources=False,
)
state, info = env.reset()